In [ ]:

import utils.common
import evaluation
import importlib
import numpy as np
import time
import models.MatrixFactorization
import models.ConstraintAutoRec
importlib.reload(utils.common)
importlib.reload(evaluation)
importlib.reload(models.MatrixFactorization)
importlib.reload(models.ConstraintAutoRec)



In [ ]:
model = models.ConstraintAutoRec.ConstraintAutoRec(utils.common.movie_lens['dimensions'], epochs=5, novelty_weight=0.1, diversity_weight=0.1)
model.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])


In [ ]:
diversity_model = models.ConstraintAutoRec.ConstraintAutoRec(utils.common.movie_lens['dimensions'], epochs=30, novelty_weight=0.25, diversity_weight=2.0)
diversity_model.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])


In [ ]:
def create_model(novelty_weight):
    m = models.ConstraintAutoRec.ConstraintAutoRec(utils.common.movie_lens['dimensions'], epochs=15, novelty_weight=novelty_weight, diversity_weight=0, batch_size=128)
    m.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])
    return m

nov_models2 = [create_model(x) for x in [0.1, 0.5, 1.0, 2.0]]

In [ ]:
import matplotlib.pyplot as plt
models

In [ ]:
#ev = evaluation.Evaluation(utils.common.movie_lens)
#big_eval = ev.evaluate_models(models)
big_eval.head()

In [ ]:
big_eval

In [ ]:
plt.bar(eval_data.index, eval_data['epc@10'])

In [ ]:
plt.plot(eval_data['novelty_weight'], eval_data['epc@10'], 'ro', ls='--')
plt.plot(eval_data['novelty_weight'], eval_data['accuracy'], 'bo', ls='--')
plt.show()

In [ ]:
def plot_bar(data, axis, x_param):
    bottom = data[axis].min() 
    top = data[axis].max()
    margin = (top - bottom) * 0.05
    fig, ax = plt.subplots()
    ax.bar(data.index, data[axis] )
    ax.set_xticks(data.index)
    ax.set_xticklabels(data[x_param])
    ax.set_ylabel(axis)
    ax.set_xlabel(x_param)
    plt.show()

In [ ]:
plot_bar(eval_data, 'epc@10', 'novelty_weight')

In [ ]:
plot_bar(eval_data, 'accuracy', 'novelty_weight')

In [4]:

import utils.common
import evaluation
import importlib
import numpy as np
import time
import models.MatrixFactorization
importlib.reload(utils.common)
importlib.reload(evaluation)
importlib.reload(models.MatrixFactorization)



<module 'models.MatrixFactorization' from '/mnt/e/owncloud/FH/MasterThesis/Code/models/MatrixFactorization.py'>

In [ ]:
# mf = models.MatrixFactorization.MatrixFactorization(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=10, latent_dim = 64)
mf.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])

In [67]:
ev = evaluation.Evaluation(utils.common.movie_lens)
res = ev.evaluate(mf)

In [ ]:
ev = evaluation.Evaluation(utils.common.movie_lens)
ev.evaluate(model)

In [ ]:
result = ev.evaluate_models([model, mf])

In [ ]:
ev.evaluate_models(nov_models2)

In [78]:
res

{'accuracy': 0.7623881019741017,
 'precision': 0.8271346642213018,
 'recall': 0.8667128559564206,
 'map@1': 0.23342581775700935,
 'map@5': 0.1375418005257009,
 'map@10': 0.11591292492586013,
 'diversity@5': 0.2578679587468785,
 'diversity@10': 0.27128682306400537,
 'epc@5': 0.7330164376135602,
 'epc@10': 0.7430216165856256,
 'epd@5': 0.3017509932890537,
 'name': 'ConstraintAutoRec',
 'dimensions': 10381,
 'latent_dims': 128,
 'accuracy_weight': 1.0,
 'novelty_weight': 0.1,
 'diversity_weight': 0.1,
 'epochs': 5,
 'batch_size': 32,
 'optimizer': 'adam'}

In [68]:
res

{'accuracy': 0.44978036950245576,
 'precision': 0.8055713264200937,
 'recall': 0.43088724952223195,
 'map@1': 0.002263434579439252,
 'map@5': 0.0008795147001557631,
 'map@10': 0.0006248166716225693,
 'diversity@5': 0.37317394044498586,
 'diversity@10': 0.37264000336748876,
 'epc@5': 0.9584747178074999,
 'epc@10': 0.9586116398686634,
 'epd@5': 0.3700744143589543,
 'name': 'MatrixFactorization',
 'latent_dim': 32,
 'epochs': 10,
 'batch_size': 128}